# The Idempotent Query Challenge

_This SQLChallenge uses the Northwind Database, which is copyright Microsoft and shared under the MIT license. To set up the database, run 01_setup-recreate Northwind.sql against a SQL Server instance dedicated to testing. Some parts of this challenge only work on SQL Server 2016 and higher._

## 1. Make this TSQL for an index create statement rerunnable

* If an index with this name exists on this table, you can assume that it has the correct definition (you do not have to check that the definition matches)
* Do not drop the index or cause it to be automatically dropped when re-run


In [13]:
USE Northwind;
GO

CREATE NONCLUSTERED INDEX ix_Customers_City_PostalCode_INCLUDES on dbo.Customers 
    (City, PostalCode) 
    INCLUDE (Region);
GO


Commands completed successfully.

: Msg 1913, Level 16, State 1, Line 3
The operation failed because an index or statistics with name 'ix_Customers_City_PostalCode_INCLUDES' already exists on table 'dbo.Customers'.

Total execution time: 00:00:00.096

### Write and test your solution here

In [14]:
USE Northwind;
GO

IF (
    SELECT COUNT(*)
    FROM sys.indexes as si 
    JOIN sys.objects as so on si.object_id = so.object_id
    JOIN sys.schemas as sc on so.schema_id = sc.schema_id
    WHERE si.name='ix_Customers_City_PostalCode_INCLUDES'
        and so.name = 'Customers'
        and sc.name = 'dbo'
) = 0
BEGIN
    CREATE NONCLUSTERED INDEX ix_Customers_City_PostalCode_INCLUDES on dbo.Customers 
        (City, PostalCode) 
        INCLUDE (Region);
END
ELSE 
BEGIN
    RAISERROR ('Index already exists', 1, 1) WITH NOWAIT
END

Commands completed successfully.

Msg 50000, Level 1, State 1, Line 19
Index already exists

Total execution time: 00:00:00.203

#### A test: does DROP_EXISTING = ON actually do anything if the index already exists and you haven't changed the definition?

In [17]:
USE Northwind;
GO

SET STATISTICS IO ON;
GO

CREATE NONCLUSTERED INDEX ix_Customers_City_PostalCode_INCLUDES on dbo.Customers 
    (City, PostalCode) 
    INCLUDE (Region) WITH (DROP_EXISTING = ON, FILLFACTOR=1);  /* beware fillfactor 1!, only doing that to inflate IO */
GO

SELECT 
    index_id, 
    so.modify_date
FROM sys.indexes as si 
JOIN sys.objects as so on si.object_id = so.object_id
WHERE si.name='ix_Customers_City_PostalCode_INCLUDES';
GO

SET STATISTICS IO OFF;
GO


Commands completed successfully.

Commands completed successfully.

Table 'Customers'. Scan count 1, logical reads 48, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

(1 row affected)

Table 'sysschobjs'. Scan count 0, logical reads 4, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

Table 'sysidxstats'. Scan count 1, logical reads 4, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

Commands completed successfully.

Total execution time: 00:00:00.032

index_id,modify_date
6,2019-06-29 12:28:36.883


___
## 2. Make this view creation statement rerunnable in two ways -- without using DROP VIEW


In [11]:
USE Northwind;
GO

CREATE VIEW [dbo].[Idempotent] AS
SELECT Orders.ShippedDate, 
    Orders.OrderID, 
    [Order Subtotals].Subtotal
FROM Orders 
INNER JOIN [Order Subtotals] ON Orders.OrderID = [Order Subtotals].OrderID
WHERE Orders.ShippedDate IS NOT NULL
GO

### 2.A First, write this in a way that will work on SQL Server 2005 and higher
* Not that you have to find a 2005 instance to test it on. Simply use a syntax that was introduced well before SQL Server 2016 SP1.
* Write your solution in a way that does not use DROP VIEW

In [20]:
USE Northwind;
GO


IF OBJECT_ID('dbo.Idempotent', 'V') IS NULL
    EXEC ('CREATE VIEW dbo.Idempotent AS SELECT 1 as col1')
GO

ALTER VIEW [dbo].[Idempotent] AS
SELECT Orders.ShippedDate, 
    Orders.OrderID, 
    [Order Subtotals].Subtotal
FROM Orders 
INNER JOIN [Order Subtotals] ON Orders.OrderID = [Order Subtotals].OrderID
WHERE Orders.ShippedDate IS NOT NULL
GO

--Extra: if you alter objects referenced in the view definition
--https://docs.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-refreshview-transact-sql
exec sp_refreshview @viewname='dbo.Idempotent';
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

### 2.B Write it in the *new* way that works in SQL Server 2016 SP1 and higher

In [23]:
USE Northwind;
GO

CREATE OR ALTER VIEW [dbo].[Idempotent] AS
SELECT Orders.ShippedDate, 
    Orders.OrderID, 
    [Order Subtotals].Subtotal
FROM Orders 
INNER JOIN [Order Subtotals] ON Orders.OrderID = [Order Subtotals].OrderID
WHERE Orders.ShippedDate IS NOT NULL
GO


--Extra: if you alter objects referenced in the view definition, you'd still need to update the metadata
--The new syntax doesn't change that 😃
exec sp_refreshview @viewname='dbo.Idempotent';
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

___
## 3. Make these DROP TABLE statements rerunnable in two ways

First, create the tables. Rerun as needed for testing.

In [29]:
USE Northwind;
GO

CREATE TABLE dbo.dropme1 (col1 int null);
CREATE TABLE dbo.dropme2 (col2 int null);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.341

Here is the non-idempotent drops to rewrite:

In [44]:
USE Northwind;
GO

DROP TABLE dbo.dropme1;

DROP TABLE dbo.dropme2;
GO

### 3.A Write this in the "classic" way which works on versions lower than SQL Server 2016

In [28]:
USE Northwind;
GO

IF OBJECT_ID('dbo.dropme1', 'U') IS NOT NULL
    DROP TABLE dbo.dropme1;

IF OBJECT_ID('dbo.dropme2', 'U') IS NOT NULL
    DROP TABLE dbo.dropme2;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.003

### 3.B Write this in the "new" way introduced in SQL Server 2016


In [30]:
USE Northwind;
GO

DROP TABLE IF EXISTS dbo.dropme1, dbo.dropme2;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.025

___
## 4. Write this set of permission statements in a rerunnable way


In [31]:
USE Northwind;
GO

CREATE ROLE NewRole AUTHORIZATION dbo;
GO

GRANT SELECT on schema::dbo TO NewRole;
GO

CREATE USER NewUser WITHOUT LOGIN;
GO

ALTER ROLE NewRole ADD MEMBER NewUser;
GO

--Validate
EXECUTE AS USER = 'NewUser';
GO
SELECT SUSER_NAME(), USER_NAME(); 

SELECT TOP 1 CompanyName
FROM dbo.Customers;
GO


REVERT;
GO


Commands completed successfully.

: Msg 15023, Level 16, State 1, Line 3
User, group, or role 'NewRole' already exists in the current database.

Commands completed successfully.

: Msg 15023, Level 16, State 1, Line 9
User, group, or role 'NewUser' already exists in the current database.

Commands completed successfully.

...

(1 row affected)

Commands completed successfully.

Total execution time: 00:00:00.241

(No column name),(No column name)
S-1-9-3-2557765982-1291698634-4099256974-3614178345,NewUser


CompanyName
Alfreds Futterkiste


### Solution goes here

In [33]:
USE Northwind;
GO

IF (SELECT DATABASE_PRINCIPAL_ID ( 'NewRole' )) IS NULL
    CREATE ROLE NewRole AUTHORIZATION dbo;
GO

GRANT SELECT on schema::dbo TO NewRole;
GO

IF (SELECT DATABASE_PRINCIPAL_ID ( 'NewUser' )) IS NULL
    CREATE USER NewUser WITHOUT LOGIN;
GO

ALTER ROLE NewRole ADD MEMBER NewUser;
GO

--Validate
EXECUTE AS USER = 'NewUser';
GO
SELECT SUSER_NAME(), USER_NAME(); 

SELECT TOP 1 CompanyName
FROM dbo.Customers;
GO


REVERT;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

(1 row affected)

Commands completed successfully.

Total execution time: 00:00:00.060

(No column name),(No column name)
S-1-9-3-2557765982-1291698634-4099256974-3614178345,NewUser


CompanyName
Alfreds Futterkiste
